# Unit 2: Fisheries Collapse Module Overview

This module will focus on examining a crucial global issue and important scientific debate about the state of global fisheries.  In this module we will seek to reproduce some of the most widely cited examples of species collapse ever, and examine the evidence behind an influential and widely cited paper on global fisheries, [Worm et al 2006](http://doi.org/10.1126/science.1132294).  However, rather than use the limited data available to Boris Worm and colleagues in 2006, we will be drawing from the best and most recent stock asssement data available today to see how those patterns have faired.  

In this module we will also begin to master one of the most important concepts in data science: manipulation of tabular data using relational database concepts. Instead of working with independent data.frames, we will be working with a large relational database which contains many different tables of different sizes and shapes, but that all all related to each other through a series of different ids.  



## The Database

We will use data from the RAM Legacy Stock Assessment Database.  In order to better introduce some important emerging technologies, we will be accessing these data directly from a relatively new platform that is now playing a key role in data sharing in machine learning communities, with the memorable name, HuggingFace.  We will be streaming data from <https://huggingface.co/datasets/cboettig/ram_fisheries/tree/main/v4.65>.  We will have more to say about this approach as we progress.



## Researcher Spotlight: Daniel Pauly

Science is done by real people.  There are many influential and colorful characters in the global fisheries debate.  I want to highlight Professor Pauly not just because he is so famous, but as an early believer in Open Science and Data Science, before we had either of those words.  His contributions in making fisheries data more open were ground breaking for their time.  I'm also indebted to Professor Pauly whom I had the privilege to meet when I was a junior scientist who had only recently released one of my first software packages, aimed at making data from FishBase more accessible. Academic researchers are typically defined by scientific publications, not software, so I was shocked that Pauly already knew of my software package, and that he encouraged me to continue developing software.  Even today that is not common advice, but I believed him, and it's probably a good reason I am where I am today.  Scientific textbooks and courses are often critiqued for failing to recognize the contributions of those from minority backgrounds, but as the texts are written on global change ecology, I think none will omit the works for Professor Pauly.




## Science Introduction

Background abbreviated documentary, features many of the leading authors on both sides https://vimeo.com/44104959

In [11]:
import ibis
from ibis import _
import ibis.selectors as s
import seaborn.objects as so



# Exercise 1: Investigating the North-Atlantic Cod

Now we are ready to dive into our data. First, We seek to replicate the following figure from the Millennium Ecosystem Assessment Project using the RAM data.

![](https://espm-157.github.io/website-r/img/cod.jpg)


In [27]:
con= ibis.duckdb.connect()

tsmetrics_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/tsmetrics.csv?download=true'
area_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/area.csv?download=true'
assessment_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/assessment.csv?download=true'
stock_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/stock.csv?download=true'
time_series_url = 'https://huggingface.co/datasets/cboettig/ram_fisheries/resolve/main/v4.65/timeseries.csv?download=true'



In [28]:


area = con.read_csv(area_url)
tsmetrics= con.read_csv(tsmetrics_url)
assessment = con.read_csv(assessment_url)
stock = con.read_csv(stock_url)
time_series = con.read_csv(time_series_url)



In [31]:
tsmetrics.filter(_.tscategory=='CATCH or LANDINGS').select(_.tslong, _.tsunique).distinct().execute()

,tslong,tsunique
0,Scientific advice for catch limit (pairs with ...,Cadvised-CMB-MT
1,Catch divided by model version only MSY,CdivMSY-mvo-ratio
2,Customary catch,CUSTC-MT
3,Recreational catch,RecC-MT
4,Total allowable catch,TAC-2-MT
...,...,...
63,Total catch. Use only when there is more than ...,TC-3-E06
64,Total catch (i.e. landings + discards. Add lan...,TC-E03
65,Total landings. Use only when there is more th...,TL-1-E03MT
66,Total landings in subarea,TL-A3-MT


In [33]:
time_series.filter(_.stockid== 'COD1ABCDE').select(_.tsid).distinct().execute()

,tsid
0,TCbest-MT
1,TL-MT
2,CdivMEANC-ratio
3,TC-MT


In [55]:
fish= (time_series
    .drop(_.stocklong)
    .rename(tsunique="tsid")
    .join(tsmetrics, "tsunique")
    .join(stock, "stockid")
    .join(assessment, "assessid")
)


cod_catch = (
    fish
    .filter(_.tscategory== "CATCH or LANDINGS")
    .filter(_.commonname == "Atlantic cod")
)

fish.group_by(_.tslong, _.tsunique).agg(n= _.count()).order_by(_.n.desc()).execute()


,tslong,tsunique,n
0,Catch divided by mean catch,CdivMEANC-ratio,138234
1,"General total Catch (TC then TL, MT units only)",TCbest-MT,127713
2,Total catch (i.e. landings + discards. Add lan...,TC-MT,86609
3,Total landings,TL-MT,72643
4,General biomass time series preferentially rel...,BdivBmgtpref-dimensionless,60802
...,...,...,...
68,Total catch (i.e. landings + discards. Add lan...,TC-2-MT,20
69,Total catch (i.e. landings + discards. Add lan...,TC-3-MT,20
70,Total allowable catch,TAC-2-MT,16
71,Total allowable catch,TAC-1-MT,15


# Excersise 2: Global Fisheries 

## Stock Collapses

We seek to replicate the temporal trend in stock declines shown in [Worm et al 2006](http://doi.org/10.1126/science.1132294):

![](https://espm-157.github.io/website-r/img/worm2006.jpg)